<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-relevant-Nielsen-products" data-toc-modified-id="Get-relevant-Nielsen-products-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get relevant Nielsen products</a></span><ul class="toc-item"><li><span><a href="#Import-purchased-products" data-toc-modified-id="Import-purchased-products-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import purchased products</a></span></li><li><span><a href="#Import-and-merge-product-data" data-toc-modified-id="Import-and-merge-product-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import and merge product data</a></span></li><li><span><a href="#Drop-nonfood-products" data-toc-modified-id="Drop-nonfood-products-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Drop nonfood products</a></span></li></ul></li><li><span><a href="#Get-nutrition-data" data-toc-modified-id="Get-nutrition-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get nutrition data</a></span><ul class="toc-item"><li><span><a href="#Assemble-nutrition-data" data-toc-modified-id="Assemble-nutrition-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Assemble nutrition data</a></span></li><li><span><a href="#Clean-nutrition-data" data-toc-modified-id="Clean-nutrition-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Clean nutrition data</a></span></li><li><span><a href="#Reshape-wide" data-toc-modified-id="Reshape-wide-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Reshape wide</a></span></li></ul></li><li><span><a href="#Impute-data-for-missings" data-toc-modified-id="Impute-data-for-missings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Impute data for missings</a></span><ul class="toc-item"><li><span><a href="#Merge-products-extra" data-toc-modified-id="Merge-products-extra-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Merge products extra</a></span></li><li><span><a href="#Impute" data-toc-modified-id="Impute-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Impute</a></span></li></ul></li><li><span><a href="#Nutrients-per-month" data-toc-modified-id="Nutrients-per-month-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Nutrients per month</a></span></li><li><span><a href="#Collapse-to-household-month-level" data-toc-modified-id="Collapse-to-household-month-level-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Collapse to household-month level</a></span><ul class="toc-item"><li><span><a href="#Import-trip-nutrition-data" data-toc-modified-id="Import-trip-nutrition-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Import trip-nutrition data</a></span></li><li><span><a href="#Get-panelists'-treatment-statuses" data-toc-modified-id="Get-panelists'-treatment-statuses-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Get panelists' treatment statuses</a></span></li><li><span><a href="#Collapse-to-household-month" data-toc-modified-id="Collapse-to-household-month-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Collapse to household-month</a></span></li><li><span><a href="#Get-panelists'-treatment-statuses" data-toc-modified-id="Get-panelists'-treatment-statuses-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Get panelists' treatment statuses</a></span></li></ul></li><li><span><a href="#Panelist-demographic-data" data-toc-modified-id="Panelist-demographic-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Panelist demographic data</a></span></li><li><span><a href="#Cook-County-Zip-Layers" data-toc-modified-id="Cook-County-Zip-Layers-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Cook County Zip Layers</a></span></li><li><span><a href="#Treated-stores" data-toc-modified-id="Treated-stores-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Treated stores</a></span></li><li><span><a href="#Alcohol-purchases-per-month" data-toc-modified-id="Alcohol-purchases-per-month-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Alcohol purchases per month</a></span></li></ul></div>

# get_nutrition_data
By: Zachary A. Goodman

This script outputs all data ready to be analyzed in `ssb_tax_analysis`. 

In [ ]:
clear all

* set working directory
qui cd ../

* set global directories
global raw_data ./data/raw_data
global temp_data ./data/temp_data
global gen_data ./data/gen_data
global figures ./tex/figures
global tables ./tex/tables

* check dependencies
which gtools

## Get relevant Nielsen products
Strategy:
1. import purchased products from 2016 - 2019 and drop duplicates
3. left merge product data
3. drop nonfood products

### Import purchased products

In [ ]:
* import all purchased upcs, keep one copy of each

clear
tempfile temp

forvalues year = 2016/2019 {
    capture restore
    preserve
    clear
    di "getting `year'..."
    import delimited "$raw_data/hms/`year'/purchases_`year'.tsv", varnames(1) colrange(2:2)
    bys upc: keep if _n == 1
    qui save `temp', replace
    restore
    append using `temp'
}

bys upc: keep if _n == 1 // why? duplicates across years. Dropping inside loop for speed

save $temp_data/unique_purchased_upcs_1619.dta, replace

In [ ]:
desc

### Import and merge product data

In [ ]:
clear
import delimited "$raw_data/hms/master/products.tsv", varnames(1) bindquote(nobind)

* keep only necessary variables
keep upc upc_ver_uc product_module_code product_group_code department_code ///
    brand_code_uc multi size1_code_uc size1_amount size1_units ///
    upc_desc brand_desc product_module_desc product_group_desc department_desc
* TODO: drop desc vars

* keep only one copy of each product
bys upc: keep if upc_ver_uc == 1
drop upc_ver_uc

In [ ]:
/* left merge product data on purchases. 
Note that all purchases have a matching product, so keeping matching
observations only is a left merge. */

merge 1:1 upc using $temp_data/unique_purchased_upcs_1619.dta, keep(3) nogen
save $temp_data/unique_purchased_products_1619.dta, replace

In [ ]:
desc

### Drop nonfood products

In [ ]:
* drop alcohol, non food grocery, and general merchandise
drop if inlist(department_code, 0, 7, 9)

* drop baby food product modules
drop if inlist(product_module_code, 1274, 1272, 1276, 1282)

* drop pet food product modules
drop if inlist(product_module_code, 1313, 1303, 1311, 1309, ///
    1306, 1300, 1310, 1304, 1299, 1301)

* drop ice, cooking wine, fruit protectors
drop if inlist(product_module_code, 2610, 1189, 1448)

* drop if unclassified non-food
drop if inlist(product_module_code, 902, 914)
drop if product_module_code == 9999 & ///
    !inlist(brand_desc, "PIRATE'S BOOTY", "JACK LINK'S", "CAJUN INJECTOR", ///
        "POP SECRET KIDZ", "PRODUCE ELECTRONIC ID BOARD", "HERSHEY'S SWEET ESCAPES", ///
        "BENE CASA", "NABISCO") & ///
    !inlist(brand_desc, "BEST FOODS", "BANQUET MORNING BAKES", "POLAR", ///
        "COUNTRY FRSH KTS&NTRLY FRSH II", "APPLE & EVE", "DUNCAN HINES", "KEURIG", ///
        "DEL MONTE", "OLE' GUACAMOLE!") & ///
    !inlist(brand_desc, "TASTY BITE", "TAYLOR FARMS & LITEHOUSE", "LUCKY ME! & SKY FLAKES")

In [ ]:
desc
count if mi(product_module_code)

In [ ]:
save $temp_data/relevant_food_products_1619.dta, replace

## Get nutrition data

Merge raw nutrition data and clean

### Assemble nutrition data

In [ ]:
* Import upc-nutrient data
clear
import delimited "$raw_data/nutrition/Nutrient.txt", varnames(1) asdouble
// only keep unprepared versions of products
keep if valuepreparedtype == "0"
drop valuepreparedtype isorcontains
qui save $temp_data/nutrition.dta, replace


* Merge nutrient labels
clear
import delimited "$raw_data/nutrition/NutrientMaster.txt", varnames(1) asdouble 
// keep selected nutrients only
keep if inlist(name, "Calories", "Calories from Fat", "Cholesterol", ///
    "Dietary Fiber", "Protein", "Saturated Fat", "Sodium", "Sugars", ///
    "Total Carbohydrate") | inlist(name, "Total Fat", "Trans Fat")
/*
Later investigate these nutrients? 
"Calcium", "Iron", "MetricServingSize", "Organic", "Vitamin A", ///
"Vitamin C", "Monounsaturated Fat", "Polyunsaturated Fat", "Potassium" 
*/
drop type
tempfile temp
qui save `temp'
use $temp_data/nutrition.dta, clear
// merge and drop nutrients that aren't in the standard nutrition facts
merge m:1 nutrientmasterid using `temp', keep(3) nogen
drop nutrientmasterid
save $temp_data/nutrition.dta, replace


* Merge all upc-level attributes
clear
import delimited "$raw_data/nutrition/ValuePrepared.txt", varnames(1) asdouble
// keep only unprepared versions of products
keep if valuepreparedtype == "0"
drop valuepreparedtype
qui save `temp', replace
import delimited "$raw_data/nutrition/Product.txt", varnames(1) asdouble clear
keep upc brand itemsize itemmeasure productweight itemname description
merge 1:1 upc using `temp', keep(3) nogen
qui save `temp', replace


* Merge upc-nutrient level data with upc-level data
use $temp_data/nutrition.dta, clear
merge m:1 upc using `temp', keep(3) nogen

* arrange data and save
rename name nutrient
sort upc nutrient
order upc nutrient quantity uom itemname description addeditem brand ///
    itemsize itemmeasure productweight servings* pct
save $temp_data/nutrition.dta, replace 


tab nutrient
unique upc
desc

### Clean nutrition data

In [ ]:
* merge with relevant UPCs to reduce necessary cleaning

use $temp_data/nutrition.dta, clear 

* drop check digit from upc and coerce to numeric
replace upc = substr(upc, 1, length(upc) - 1)
destring upc, replace

/* after throughly checking the data, negative signs in front of nutrient
quantities are unintentional. */
* remove the negative signs
replace quantity = -1*quantity if quantity < 0
replace pct = -1*pct if pct < 0

* drop if missing quantity
drop if mi(quantity)

* merge with Nielsen product upcs from above
merge m:1 upc using $temp_data/unique_purchased_upcs_1619.dta, keep(2 3)

* save unmatched relevant products for later imputation
preserve
keep if _m == 2
keep upc
save $temp_data/unique_purchased_upcs_tobeimputed_1619.dta, replace
restore

* proceed with matching upcs only
keep if _m == 3
drop _m

In [ ]:
* rename nutrient labels
replace nutrient = subinstr(nutrient, " ", "", .)
replace nutrient = lower(nutrient)
replace nutrient = "fiber" if nutrient == "dietaryfiber"
replace nutrient = "carbs" if nutrient == "totalcarbohydrate"
replace nutrient = "satfat" if nutrient == "saturatedfat"
replace nutrient = "fat" if nutrient == "totalfat"

tab nutrient

In [ ]:
* fix uom errors - all corrections were manually checked after looking up the products online
tab uom
replace uom = "g" if inlist(uom, "0", "4", "G", "Null", "g ", "g0", "gf", "m")
replace quantity = 13 if uom == "4g"
replace uom = "g" if uom == "4g"
replace quantity = 0.4 if uom == "9"
replace uom = "g" if uom == "9"
replace quantity = 14 if uom == "oz" & quantity == .48
replace uom = "g" if uom == "oz"
replace quantity = 0 if uom == "trace"
replace uom = "g" if uom == "trace"

* put all nutrition units in grams
replace quantity = quantity/1000 if inlist(uom, "mg", "mg ", "mEq", "Mg")
replace uom = "g" if inlist(uom, "mg", "mg ", "mEq", "Mg")

* fix absurdly high sodium
// li itemname quantity pct if quantity > 2 & pct < 60 & nutrient == "sodium"
replace quantity = quantity/1000 if quantity > 2 & pct < 60 & nutrient == "sodium"

tab uom

* those missing uom are calories, calories from fat, 0, or grams without label

In [ ]:
* use pct flag to check other nutrients
* check pct against FDA's DRV for macros
program gen_pctcheck
    capture drop pctcheck
    gen pctcheck = quantity/2.75 if nutrient == "carbs" 
    replace pctcheck = quantity/.03 if nutrient == "cholesterol" 
    replace pctcheck = quantity/.78 if nutrient == "fat"
    replace pctcheck = quantity/.28 if nutrient == "fiber"
    replace pctcheck = quantity/.50 if nutrient == "protein"
    replace pctcheck = quantity/.20 if nutrient == "satfat"
    replace pctcheck = quantity/.023 if nutrient == "sodium"
    replace pctcheck = quantity/.50 if nutrient == "sugars"
    end

gen_pctcheck

In [ ]:
* get nutrition info for products whose pct are wildly off

preserve
keep if !mi(pctcheck, pct) & pctcheck/10 > pct & pct > 0
unique upc
export excel $temp_data/uom_needs_corrections_202103.xlsx, firstrow(var) replace
restore

In [ ]:
* append corrected data

drop if !mi(pctcheck, pct) & pctcheck/10 > pct & pct > 0
preserve
import excel $temp_data/uom_corrections.xlsx, firstrow clear
tempfile temp
qui save `temp'
restore
append using `temp'

In [ ]:
* check nutrition info for products with high percent RDV

gen_pctcheck

preserve
keep if pctcheck > 100 & !mi(pctcheck)
unique upc
tab nutrient
sort nutrient pctcheck
export excel $temp_data/uom_needs_checking.xlsx, firstrow(var) replace
restore

In [ ]:
* append corrected data

drop if pctcheck > 100 & !mi(pctcheck)
preserve
import excel $temp_data/uom_needs_checking_corrected.xlsx, firstrow clear
tempfile temp
qui save `temp'
restore
append using `temp'

In [ ]:
* delete duplicate entries
* This can happen if products have nutrition facts for multiple serving sizes,
*  e.g. regular serving size and 100g
* We only need one of the two to get total grams.

bys upc nutrient: gen N = _N
preserve
keep if N > 1
drop N
sort upc description itemsize itemmeasure productweight servingsizetext servingsizeuom nutrient quantity
export excel $temp_data/uom_duplicates_202103.xlsx, firstrow(var) replace
restore

// drop if N > 1
preserve
import excel $temp_data/uom_duplicates_corrected.xlsx, firstrow clear
tempfile temp
qui save `temp'
restore
//drop N
append using `temp', force

In [ ]:
* Clean servings per container

// sort upc servingspercontainer
// by upc: gen spc2 = servingspercontainer[_N]
// replace servingspercontainer = spc2 if mi(servingspercontainer)

preserve
keep servingspercontainer
bys servingspercontainer: keep if _n == 1
export excel $temp_data/spc_unique.xlsx, firstrow(var) replace
restore

* merge corrections

preserve
import excel $temp_data/spc_unique_corrected.xlsx, firstrow clear
tempfile temp
qui save `temp'
restore

merge m:1 servingspercontainer using `temp', nogen keep(1 3)
drop servingspercontainer
replace spc = . if spc == 0

### Reshape wide
Save data at the upc level instead of upc-nutrient

In [ ]:
restore
preserve

In [ ]:
* check uniqueness
unique upc nutrient

// * drop unnecessary vars
drop uom addeditem pct* N

// * rename quantity to grams
rename quantity grams

In [ ]:
* fix non-uniqueness within UPC

bys upc: gen N = _N
bys upc productweight: gen needs_fix = _N < N
foreach v in servingsizetext servingsizeuom spc {
    bys upc `v': replace needs_fix = _N < N if needs_fix == 0
}

tab needs_fix

replace productweight = 293.8 if needs_fix & N == 6
replace servingsizetext = "1" if needs_fix & N == 6
replace servingsizeuom = "can" if needs_fix & N == 6
replace spc = 7 if needs_fix & N == 11

In [ ]:
drop N needs_fix

* reshape
reshape wide @grams, i(upc) j(nutrient, string)
rename (caloriesgrams caloriesfromfatgrams sugarsgrams) (calories caloriesfromfat sugargrams)

In [ ]:
desc

In [ ]:
* save data
save $temp_data/unique_upc_nutrients_202103.dta, replace

## Impute data for missings

The imputation process is as follows (similar to Dubois, Griffith, and Nevo (2014) AER):
1. Direct UPC match
2. Impute within same product module, size type, product, brand, flavor, variety, type, formula, and style
3. Loosen brand restriction
4. Loosen flavor, variety, type, formula, style restrictions
5. Loosen product restriction
6. Manually impute 

In [ ]:
/*
UPCs with Nielsen product descriptors: $raw_data/hms/master/products.tsv
UPCs purchased: $temp_data/unique_purchased_upcs_1619.dta
UPCs with nutrition: $temp_data/unique_upc_nutrients_202103.dta
UPCs relevant: $temp_data/relevant_food_products_1619.dta

Strategy:
- Start with products.tsv
- keep purchased UPCs
- merge nutrient matches
- keep matches and unmatched-relevant 
- impute
- drop irrelevant matches

*/

clear
import delimited "$raw_data/hms/master/products.tsv", varnames(1) bindquote(nobind)

* keep only one copy of each product
bys upc: keep if upc_ver_uc == 1
drop upc_ver_uc

* keep only relevant UPCs
merge 1:1 upc using $temp_data/unique_purchased_upcs_1619.dta, keep(3) nogen


// * keep only necessary variables
// keep upc upc_ver_uc product_module_code product_group_code department_code ///
//     brand_code_uc multi size1_code_uc size1_amount size1_units ///
//     upc_desc brand_desc product_module_desc product_group_desc department_desc
// * TODO: drop desc vars

* merge nutrient matches
merge 1:1 upc using $temp_data/unique_upc_nutrients_202103.dta
gen has_nutrition = _m == 3 
assert _m != 2  // shouldn't have any _m == 2
drop _m

* merge relevant UPCs, keep relevant or has_nutrition
merge 1:1 upc using $temp_data/relevant_food_products_1619.dta
assert _m != 2  // shouldn't have any _m == 2
gen relevant = _m == 3
keep if has_nutrition | relevant
drop _m

In [ ]:
* clean up vars

* drop if RMS only 
drop if dataset_found_uc == "RMS"
drop dataset_found_uc

* store brand identifier
gen storebrand = regexm(upc_descr, "CTL BR")
replace storebrand = regexm(brand_descr, "CTL BR") if storebrand == 0
tab has_nutrition storebrand, col

* drop if not food (but kept becuase has_nutrition)
drop if inlist(department_code, 0, 7, 8, 9)

* drop reference card (see todos below):
drop if department_code == 99
drop if regexm(product_module_descr, "REFERENCE CARD")
drop if regexm(product_module_descr, "RBC")

* drop nonfood product groups
drop if inlist(product_group_descr, "ICE", "PET FOOD")

* TODO: later, keep reference card goods to look at intensive margin
* TODO: keep alcohol and look at effects on extensive and intensive margins

In [ ]:
tab department_descr has_nutrition, row

In [ ]:
tab product_group_descr has_nutrition

In [ ]:
tab product_module_descr has_nutrition if product_group_descr == "FRESH PRODUCE"

* TODO: map USDA data to produce 

### Merge products extra

We need the flavor code, etc. for imputing

In [ ]:
preserve

In [ ]:
* Loop over each year, keep most recent variant, and only if in list of upcs

tempfile temp

forvalues year = 2016/2019 {
    import delimited "$raw_data/hms/`year'/products_extra_`year'.tsv", varnames(1) clear
    keep if upc_ver_uc == 1
    keep upc panel_year product* flavor* variety* type* formula* style* form*
    if `year' != 2016 { 
        append using `temp'
    }
    qui save `temp', replace 
}

save $temp_data/products_extra_1619.dta, replace

In [ ]:
* keep most recent upc

use $temp_data/products_extra_1619.dta, clear

* most most recent year
sort upc panel_year
by upc: keep if _n == _N
unique upc

* merge with preserved UPCs
tempfile temp
qui save `temp'
restore
merge 1:1 upc using `temp', keep(1 3) nogen

In [ ]:
* clean vars

* rename vars we'll reference often
rename brand brand_syn
rename (product_module_code size1_units product_code brand_code_uc flavor_code variety_code ///
        type_code formula_code style_code) ///
    (pm size product brand flavor variety type formula style)

foreach v in product flavor variety type formula style {
    replace `v' = . if inlist(`v'_descr, "NOT STATED", ///
        "NOT APPLICABLE", "NA", "N/A")
}

### Impute

`impute`:
- 0 = not yet imputed
- 1 = direct match
- 2 = same PM, brand, etc.
- etc. from above


In [ ]:
* Impute = 1 -> direct match

gen imputed = has_nutrition

In [ ]:
* impute function

capture program drop impute_byid
program define impute_byid
    args id round

    * calculate mean serving ratio within group
    capture drop servingratio meanservingratio
    gen servingratio = spc / size1_amount
    bys `id': egen meanservingratio = mean(servingratio)

    * impute each nutrient within group
    local nutrients calories caloriesfromfat carbs fat fiber protein ///
        satfat sodium sugar transfat
    foreach nt of varlist `nutrients' {
        di "Imputing `nt'..."
        qui bysort `id': egen meannt = mean(`nt')
        qui replace imputed = `round' if mi(`nt') & !mi(meannt) & imputed == 0
        replace `nt' = meannt if mi(`nt') & !mi(meannt)
        drop meannt
    }

    * impute servings per container
    replace spc = meanservingratio * size1_amount if imputed == `round'

    end

In [ ]:
* Impute = 2: within pm, size type, brand, product, flavor, etc.

capture drop groupid
egen groupid = group(pm size product brand flavor variety type formula style), missing

impute_byid groupid 2

In [ ]:
* Impute = 3: relax brand

capture drop groupid
egen groupid = group(pm size product flavor variety type formula style), missing

impute_byid groupid 3

In [ ]:
* Impute = 4: relax flavor, variety, ...

capture drop groupid
egen groupid = group(pm size product), missing

impute_byid groupid 4

In [ ]:
* Impute = 5: product

capture drop groupid
egen groupid = group(pm size), missing

impute_byid groupid 5

In [ ]:
tab imputed

In [ ]:
* save imputed nutrition data
save $temp_data/imputed_nutrition_1619.dta, replace

In [ ]:
use $temp_data/imputed_nutrition_1619.dta, clear

In [ ]:
* export products missing any nutrition to csv for manual impute

capture restore
preserve

capture drop temp
gen temp = mi(calories, caloriesfromfat, carbs, fat, fiber, protein, ///
        satfat, sodium, sugar, transfat)
tab temp

keep if temp
export excel $temp_data/nutrient_manual_impute.xlsx, firstrow(var) replace

restore

In [ ]:
/* bring back manually imputed data
Had to remove:
- Nonfood
- Generic food kits without descriptors
- Gift packages that have candy in them
*/

drop if mi(calories, caloriesfromfat, carbs, fat, fiber, protein, ///
        satfat, sodium, sugar, transfat)
capture restore
preserve
tempfile temp
import excel $temp_data/nutrient_manual_impute_corrected.xlsx, firstrow clear
qui save `temp'
restore
append using `temp'

In [ ]:
* update imputed nutrition data
//save $temp_data/imputed_nutrition_1619.dta, replace

##  Nutrients per month

In this section, we do the following:
1. Divide the panel into batches
2. Load trips for a given batch, merging on household_cd (9.4M lines/year)
3. Merge purchases on trip_code_uc (64M lines/year)
4. Merge nutrition on UPC
5. Keep relevant nutrition variables
6. Collapse to trip level, taking sums over nutrition
7. Save batch and repeat

In [ ]:
* create a frame for nutrition

cwf default
capture frame drop nutrition
frame create nutrition
cwf nutrition
use $temp_data/imputed_nutrition_1619.dta

* keep only necessary vars
keep upc multi calories-transfatgrams imputed storebrand

frames dir

In [ ]:
* create a frame for products

cwf default
capture frame drop products
frame create products
cwf products
import delimited $raw_data/hms/master/products.tsv, varnames(1) bindquote(nobind) clear

* keep only necessary obs
keep if inlist(department_code, 1, 2, 3, 8)
keep if upc_ver_uc == 1

* keep only necessary cols
keep upc multi department_desc product_group_desc product_module_descr size1_amount size1_units

* gen volumes for certain product properties
gen oz_alcohol = size1_amount if department_desc == "ALCOHOLIC BEVERAGES" & size1_units == "OZ"
replace oz_alcohol = size1_amount * 33.814 if mi(oz_alcohol) & ///
     department_desc == "ALCOHOLIC BEVERAGES" & size1_units == "LI"
replace oz_alcohol = size1_amount * 0.033814 if mi(oz_alcohol) & ///
     department_desc == "ALCOHOLIC BEVERAGES" & size1_units == "ML"
gen oz_coffee_fluid = size1_amount if product_module_desc == "COFFEE - LIQUID" & size1_units == "OZ"
gen oz_coffee_solid = size1_amount if size1_units == "OZ" & product_module_desc == "GROUND AND WHOLE BEAN COFFEE"
gen oz_frozen_juice = size1_amount if product_group_desc == "JUICES, DRINKS-FROZEN" & size1_units == "OZ"
gen oz_fruit_punch = size1_amount if product_module_desc == "FRUIT PUNCH BASES & SYRUPS" & size1_units == "OZ"
gen oz_milk = size1_amount if size1_units == "OZ" & ///
    inlist(product_module_desc, "DAIRY-MILK-REFRIGERATED", "DAIRY-FLAVORED MILK-REFRIGERATED")
gen oz_soda_diet = size1_amount if size1_units == "OZ" & product_module_desc == "SOFT DRINKS - LOW CALORIE"
gen oz_soda_regular = size1_amount if size1_units == "OZ" & product_module_desc == "SOFT DRINKS - CARBONATED"
gen oz_soda_syrup = size1_amount if size1_units == "OZ" & product_module_desc == "FOUNTAIN BEVERAGE SYRUP"
gen oz_tea_liquid = size1_amount if size1_units == "OZ" & product_module_desc == "TEA - LIQUID"
gen oz_water_bottled = size1_amount if size1_units == "OZ" & product_module_desc == "WATER-BOTTLED"

* powdered soft drinks measured in different units
gen oz_soft_drinks_powder = size1_amount * 32 if size1_units == "QT" & product_module_desc == "SOFT DRINKS - POWDERED"

* drop superfluous vars
drop department_desc product_group_desc product_module_descr size1_amount size1_units
gen dontdrop = 0
foreach v of varlist oz_alcohol-oz_soft_drinks_powder {
    replace dontdrop = 1 if !mi(`v')
}
keep if dontdrop
drop dontdrop

* rename mutli for merge
rename multi multi2

In [ ]:
* write function to iteratively merge nutrition on purchases

capture program drop agg_batch
program agg_batch
    args batch_name

    * get batchsize
    cwf trips
    qui sum trip_code_uc
    local batchsize = ceil(r(N) / 10)
    local topmax = r(N)

    * loop over batches
    forvalues b = 1/10 {
        
        di "Batch #`b'..."
        local bottom = 1 + (`b' - 1) * `batchsize'
        local top = min(`b' * `batchsize', `topmax')
        
        * keep subset of trips
        cwf default
        capture frame drop trips_sub
        frame copy trips trips_sub
        cwf trips_sub
        keep in `bottom'/`top'
                
        * copy nutrition, keep subset
        capture frame drop purchases_sub
        frame copy purchases purchases_sub
        cwf purchases_sub
        frlink m:1 trip_code_uc, frame(trips_sub)
        frget household_code, from (trips_sub)
        keep if !mi(household_code)
        
        * merge nutrition
        frlink m:1 upc, frame(nutrition)
        frget *, from(nutrition)
        
        * merge products
        frlink m:1 upc, frame(products)
        frget *, from(products)
        replace multi = multi2 if mi(multi)
        drop multi2
    
        * generate additional vars
        gen items_scanned = 1 * quantity
        gen items_food = !mi(impute) * quantity
        gen items_coupons = coupon_value > 0 * quantity
        gen final_price_paid = total_price_paid - coupon_value
        gen final_price_paid_food = final_price_paid * (items_food > 0)
        rename coupon_value coupons_amount_saved
        rename deal_flag items_deals
        rename storebrand items_storebrand
        replace items_deals = items_deals * quantity
        replace items_storebrand = items_storebrand * quantity
        
        * add grams of sugar per product type
        gen beverage = 0
        local namelist = "coffee_fluid coffee_solid frozen_juice fruit_punch milk soda_diet soda_regular soda_syrup tea_liquid soft_drinks_powder"
        foreach n in `namelist' {
            qui gen sugar_`n' = (oz_`n' > 0) * sugargrams
            qui replace beverage = 1 if oz_`n' > 0
        }
        qui gen sugar_nonbeverage = sugargrams * (1 - beverage)
        
        * Prepare vars for collapsing
        * multiply relevant vars by quantity
        foreach var of varlist calories-transfatgrams oz_alcohol-oz_soft_drinks_powder sugar_* {
            qui replace `var' = `var' * multi * quantity
        }
        
        * imputed
        forvalues i = 0/5 {
            gen imputed`i' = (imputed == `i') * quantity
        }

        * keep relevant vars
        local collapsevars items_* final_price_paid* ///
            coupons_amount_saved imputed* ///
            calories-transfatgrams oz_alcohol-oz_soft_drinks_powder sugar_*
        keep household_code trip_code_uc `collapsevars'
        drop imputed
        
        * Next, collapse to trip-level taking sums
        collapse (sum) `collapsevars', by(household_code trip_code_uc)
        
        * Save data
        save $temp_data/`batch_name'_`b'.dta, replace
    }

    * append across batches
    * current batch is 10
    di "Appending all batches..."
    forvalues b = 1/9 {
        append using $temp_data/`batch_name'_`b'.dta
    }

    save $gen_data/`batch_name'.dta, replace
    
    end

In [ ]:
* loop over years

forvalues year = 2016/2019 {
    
    di "Getting data for year = `year'..."
    
    * trips frame
    cwf default
    capture frame drop trips
    frame create trips
    cwf trips
    import delimited "$raw_data/hms/`year'/trips_`year'.tsv", varnames(1)
    
    * purchases frame
    capture frame drop purchases
    frame create purchases
    cwf purchases
    import delimited "$raw_data/hms/`year'/purchases_`year'.tsv", varnames(1) clear
    drop upc_ver_uc
    
    * run agg function
    agg_batch nutrition_trips_`year'

}

In [ ]:
unique household_code

In [ ]:
unique trip_code_uc

In [ ]:
sum

## Collapse to household-month level

In [ ]:
* Define function to collapse to trip level

capture program drop collapse_trip
program collapse trip
    args year
    
    * import trips-nutrition
    cwf default
    use $gen_data/nutrition_trips_`year'.dta, clear

    * merge trips
    capture frame drop trips
    frame create trips
    cwf trips
    import delimited "$raw_data/hms/`year'/trips_`year'.tsv", varnames(1)
    * TODO keep store_zip3, method_of_payment_cd
    * TODO add indicator for treated store
    * TODO anything with method of payment? Perhaps SNAP indicator
    keep trip_code_uc purchase_date total_spent
    cwf default
    frlink m:1 trip_code_uc, frame(trips)
    frget *, frame(trips)

    * gen month indicator
    gen date = date(purchase_date, "YMD")
    format %td date
    gen month = month(date)
    gen year = year(date)

    * drop vars that do not remain constant
    drop date purchase_date date trip_code_uc trips store_zip3 method_of_payment_cd

    * gen count var for shopping trips
    gen trips = 1

    * collapse taking sums
    di "Collapsing..."
    gcollapse (sum) items_deals-sugar_nonbeverage trips total_spent, by(household_code year month)

    * save output
    save $temp_data/

    end

### Import trip-nutrition data

In [ ]:
* Import trips data containing nutrition per trip

cwf default
use $gen_data/nutrition_trips_2016.dta, clear
forvalues year = 2017/2019 {
    append using $gen_data/nutrition_trips_`year'.dta
}

// sum

In [ ]:
* Merge trips (to get purchase_date, store_zip3, total_spent, method_of_payment_cd)

capture frame drop trips
frame create trips
cwf trips

forvalues year = 2016/2019 {
    preserve
    clear
    import delimited "$raw_data/hms/`year'/trips_`year'.tsv", varnames(1)
    tempfile temp
    qui save `temp'
    restore
    append using `temp'
}

keep trip_code_uc purchase_date store_zip3 total_spent method_of_payment_cd

// sum

In [ ]:
* link trips to purchases and copy relevant trips info

cwf default
frlink 1:1 trip_code_uc, frame(trips)
frget purchase_date store_zip3 total_spent method_of_payment_cd, from(trips)

save $gen_data/nutrition_trips_all.dta, replace

### Get panelists' treatment statuses

In [ ]:
* merge panelists on trips

* switch frames
cwf default

* get year
gen date = date(purchase_date, "YMD")
format %td date
gen year = year(date)

* link panelists
frlink m:1 household_code year, frame(panelists)
frget projection_factor panelist_zipcd dma_cd fips_state_cd fips_county_cd, from(panelists)
sum

* save subset
save $gen_data/panelist_nutrition_trips_dma, replace

In [ ]:
* merge panelists on trips

* save panelists as temporary data
cwf panelists
tempfile temp
qui save `temp'

* switch frames
cwf default

* get year
gen date = date(purchase_date, "YMD")
format %td date
gen year = year(date)

* merge panelists
merge m:1 household_code year using `temp', keep(3) nogen
sum

* save subset
save $gen_data/panelist_nutrition_trips_dma, replace

### Collapse to household-month 

In [ ]:
* Caution, this takes an hour or so

* resume from prev
di "Reading data..."
cwf default
use $gen_data/nutrition_trips_all.dta, clear

* gen month indicator
gen date = date(purchase_date, "YMD")
format %td date
gen month = month(date)
gen year = year(date)

* TODO add indicator for treated store
* TODO anything with method of payment? Perhaps SNAP indicator

* drop vars that do not remain constant
drop date purchase_date date trip_code_uc trips store_zip3 method_of_payment_cd

* gen count var for shopping trips
gen trips = 1

* collapse taking sums
di "Collapsing..."
gcollapse (sum) items_deals-sugar_nonbeverage trips total_spent, by(household_code year month)

In [ ]:
save $gen_data/hh_year_month.dta

### Get panelists' treatment statuses

In [ ]:
* Import raw panelists and add treatment info

frame create panelists
cwf panelists

forvalues year = 2016/2019 {
    
    preserve
    import delimited "$raw_data/hms/`year'/panelists_`year'.tsv", varnames(1) clear
    
    * keep only relevant vars (we'll adjust this as needed for control variables)
    keep household_cd projection_factor panel_year panelist_zipcd dma_cd fips_state_cd fips_county_cd
    rename household_cd household_code
    
    * append to tempfile
    tempfile temp
    qui save `temp'
    
    restore
    append using `temp'
}

rename panel_year year
unique household_code
unique household_code year


* get treatment identifiers
qui do ./jupyter/insample.do
tab year dma_treated

In [ ]:
* switch frames
cwf default
use $temp_data/hh_year_month.dta, clear
drop if year == 2015 // late december purchases

* link panelists
frlink m:1 household_code year, frame(panelists)
qui frget *, from(panelists)

* handle late december purchases
replace year = year + 1 if mi(locality) // late december purchases
drop panelists-locality
frlink m:1 household_code year, frame(panelists)
frget *, from(panelists)
drop panelists
// sum

In [ ]:
* Add some additional variables

gen yearmonth = year + (month - 1) / 12

* treatment dates
gen yearmonth_treat = .
replace yearmonth_treat = 2017 + (4 - 1) / 12 if locality == "Albany"
replace yearmonth_treat = 2015 + (3 - 1) / 12 if locality == "Berkeley"
replace yearmonth_treat = 2017 + (7 - 1) / 12 if locality == "Boulder"
replace yearmonth_treat = 2017 + (8 - 1) / 12 if locality == "Cook"
replace yearmonth_treat = 2017 + (7 - 1) / 12 if locality == "Oakland"
replace yearmonth_treat = 2017 + (1 - 1) / 12 if locality == "Philly"
replace yearmonth_treat = 2018 + (1 - 1) / 12 if locality == "San Francisco"
replace yearmonth_treat = 2018 + (1 - 1) / 12 if locality == "Seattle"

* locality
encode locality, gen(locality_num)
gen evertreated = locality != "Control"

* tax amount
gen tau = 0
replace tau = 1 if yearmonth >= yearmonth_treat & inlist(locality, "Albany", "Berkeley", "Cook", "Oakland", "San Francisco")
replace tau = 0 if locality == "Cook" & yearmonth > 2017 + (12 - 1.01) / 12
replace tau = 2 if yearmonth >= yearmonth_treat & locality == "Boulder"
replace tau = 1.5 if yearmonth >= yearmonth_treat & locality == "Philly"
replace tau = 1.75 if yearmonth >= yearmonth_treat & locality == "Seattle"

* months since treatment
gen months_since_treat = round((yearmonth - yearmonth_treat) * 12, 1)

In [ ]:
* drop panelists who move

capture drop N
bys household_code locality: gen N = 1 if _n == 1
by household_code: ereplace N = sum(N)
tab N

keep if N == 1
drop N

In [ ]:
* save data

save $gen_data/panelist_nutrition_month_prepped, replace


* export to CSV for use in R

export delimited $gen_data/panelist_nutrition_month.csv, replace

## Panelist demographic data

Produce one file that will merge m:1 with panelist-month purchase data.

Control vars a la Harding et al 2012 AEJ: EP

>We measure...in this manner because women are more likely to be the primary shoppers in a two-headed household, but due to high correlation between male and female head's...levels, our estimates are not sensitive to how we measure...

In [ ]:
cwf default
capture frame drop panelists
frame create panelists
cwf panelists

forvalues year = 2016/2019 {
    
    preserve
    import delimited "$raw_data/hms/`year'/panelists_`year'.tsv", varnames(1) clear
    
    * rename vars
    rename (household_cd panel_year) (household_code year)
    
    * append to tempfile
    tempfile temp
    qui save `temp'
    
    restore
    append using `temp'
}

In [ ]:
* 1 - 6+ HH members
gen hh_size = household_size
replace hh_size = 6 if hh_size > 6
label var hh_size "Household Size, top-coded at 6"

* Low, middle, high income
gen hh_income = 1 
replace hh_income = 2 if inrange(household_income, 17, 21)
replace hh_income = 3 if inrange(household_income, 23, 26)
replace hh_income = 4 if household_income == 27
label var hh_income "Household Income"
label define income 1 "<$35K" 2 "$35K - $59,999" ///
    3 "$60K - $99,999" 4 ">$100K"
label values hh_income income

* (Female) Age < 35, 35 - 49, 50 - 64, 65+
gen hh_age = female_head_age
replace hh_age = male_head_age if hh_age == 0
replace hh_age = -1*hh_age
replace hh_age = 4 if hh_age == -9
replace hh_age = 3 if hh_age <= -7
replace hh_age = 2 if hh_age <= -4
replace hh_age = 1 if hh_age < 0
label var hh_age "Age of the (female) head of household"
label define age 1 "<35" 2 "35 - 49" ///
    3 "50-64" 4 "65+"
label values hh_age age

* <HS, HS grad, some college, BA+
gen hh_educ = female_head_educ
replace hh_educ = male_head_educ if hh_educ == 0 
replace hh_educ = hh_educ - 1 if hh_educ != 1
replace hh_educ = 4 if hh_educ == 5
label var hh_educ "Education of the (female) head of household"
label define educ 1 "< HS" 2 "HS Grad" ///
    3 "Some College" 4 "BA+" 
label values hh_educ educ

* White, Black, Hispanic, Asian
rename race hh_race 
label var hh_race "Racial identity of the household"
label define race 1 "White" 2 "Black" 3 "Asian" 4 "Other"
label values hh_race race

* Kids under 18 y/n
gen hh_child = 0
replace hh_child = 1 if age_and != 9
label var hh_child "Indicates if any children < 18 in HH"
label define child 1 "Yes" 2 "No"
label values hh_child child

* Female <30 hrs, 30 - 34 hrs, >= 35 hrs, not employed (includes retired)
rename female_head_emp hh_emp_female
label var hh_emp_fem "Hours employment/week of female head of HH"
label define emp 1 "< 30" 2 "30 - 34" 3 ">= 35" 9 "Not employed" 0 "No head of this gender"
label values hh_emp_fem emp

* Male <30 hrs, 30 - 34 hrs, >= 35 hrs, not employed (includes retired)
rename male_head_emp hh_emp_male
label var hh_emp_male "Hours employment/week of male head of HH"
label values hh_emp_male emp

* Presence of Female head - captured by hh_female_emp

* Presence of Male head - captured by hh_male_emp

In [ ]:
* keep vars and save

keep household_code year projection_factor ///
    hh_*

//desc

save $gen_data/panelist_demographics.dta

## Cook County Zip Layers

In [ ]:
use $gen_data/panelist_nutrition_month_prepped, clear
keep if dma_cd == 602

In [ ]:
desc

## Treated stores

Stores have zip3 and not zip5, so we will impute using modal customer's zip code.

In [ ]:
* Import trips data, then collapse to store level

clear
use $raw/2017_hms/trips_2017.dta
* TODO: merge other years?

merge m:1 household_code using $raw/2017_hms/panelists_2017.dta, nogen
keep trip_code_uc store_zip3 panelist_zip_code total_spent

* get trips by store-panelist_zip
gen trips = 1
collapse (sum) trips total_spent, by(store_code_uc store_zip3 panelist_zip_code)

* Assign each store the modal zip, tie goes to larger sales
sort store_code_uc trips total_spent
by store_code_uc: gen store_zip5 = panelist_zip_code if _n == 1
by store_code_uc: ereplace store_zip5 = min(store_zip5)

* save data


## Alcohol purchases per month

In [ ]:
TODO